# Open source quantum development with Codespaces

This notebook walks through a simple demonstration of how to go from user developed quantum programs to submitting them to be run on cloud resources, all from Python and Jupyter Notebooks. 

All the pieces of the demo are connected by [QIR], or quantum intermediate representation, an open specification maintained by the [QIR Alliance].
QIR is related to LLVM in that it re-uses a lot of the LLVM spec and infrastructure so most of the standard optimizations and utilities work out of the box for quantum programs.

**About the demo**

The basic workflow starts in [QuTiP], a Python package that allows you to express quantum programs in a convinent way.
From there, we will compile the program to QIR from a buil-in QuTiP function that using [PyQIR], a QIR Alliance project.
Once programs are compiled to [QIR], we will us [qat] to run optimization passes on the QIR bitcode.
Lastly, we will send the QIR program to the Azure Quantum service to be executed on quantum hardware.

![](stack.png)

*More details about the repo structrure and contents can be found in the [README](README.md).*

---

## 1. Develop quantum programs in Python

There are a wide variety of languages and frameworks that the OSS community is using to developing quantum progams.
Many of them either directly or via other OSS tools can compile to QIR, you can see a list of options in the [README](README.md) of this repo.
Here we will use [QuTiP](https://qutip.org), the open source quantum toolbox for Python.

_Learn more about quantum programming with the [QuTiP tutorials](https://qutip.org/tutorials#quantum-information-processing)._

Lets start with a simple teleportation quantum program:

In [ ]:
from qutip_qip.circuit import QubitCircuit


In [ ]:

# Ask device for some qubits, and create a program to manipulate them
program = QubitCircuit(3, num_cbits=2)
msg, here, there = range(3)

# Encode message
program.add_gate("RZ", targets=[msg], arg_value=0.123)

# Entangle qubits
program.add_gate("SNOT", targets=[here])
program.add_gate("CNOT", targets=[there], controls=[here])
program.add_gate("CNOT", targets=[here], controls=[msg])
program.add_gate("SNOT", targets=[msg])

# Measure qubits
program.add_measurement("Z", targets=[msg], classical_store=0)
program.add_measurement("Z", targets=[here], classical_store=1)

# Make final operations based on measurement results
program.add_gate("X", targets=[there], classical_controls=[0])
program.add_gate("Z", targets=[there], classical_controls=[1])

# Inspect the quantum program
program.gates

---

## 2. Compile programs to QIR

For more info on the QIR export in QuTiP, check out the [docs](https://qutip-qip.readthedocs.io/en/latest/qip-simulator.html?highlight=qir#import-and-export-quantum-circuits).

In [ ]:
from qutip_qip.qir import circuit_to_qir

In [ ]:
with open('output/qutip_program.bc', 'wb') as file:
    file.write(circuit_to_qir(program, "bitcode"))  # type: ignore
    
with open('output/qutip_program.ll', 'w') as file:
    file.write(circuit_to_qir(program, "text"))  # type: ignore

Great! we now have a way of lowering high-level user code to QIR, but what does that look like?
Let's check out part of the QIR file we just generated:

In [ ]:
with open('output/qutip_program.ll', 'r') as f:
    print(f.read())

**NOTE:** PyQIR is a dependency for this QuTiP function, which calls the `pyqir-generator` tool to walk the quantum program and build up the LLVM bitcode. The core rust crate `qirlib` that the PyQIR package is built around can be found on [GitHub](https://github.com/qir-alliance/pyqir/tree/main/qirlib), and used separately to write quantum programs in Rust!

---

## 3. Optimize programs

This isn't the nicest way to debug things (though really it should be only the compiler looking). Let's look at a control flow graph:

In [ ]:
!opt -dot-cfg output/qutip_program.ll -o output/qutip_program.dot && mv .main.dot output/qutip_program.dot

In [ ]:
!dot -Tpng output/qutip_program.dot -o output/qutip_program.png

In [ ]:
!qat --apply --target-def ./qat_config.yml -S output/qutip_program.ll -o output/qutip_program_optimized.ll

In [ ]:
import os

from numpy import size
print(f'Before: {os.stat("output/qutip_program.ll").st_size}, {os.stat("output/qutip_program_optimized.ll").st_size}')

---

## 4. a) Submitting quantum programs to the cloud

Now that we have generated our qir file, we want to submit it to a cloud provider to run it against a simulator or hardware device.

We start by importing the appropriate Python package to work connect to an Azure Quantum resource.

In [ ]:
from azure.identity import AzureCliCredential

NOTE: You will have to open a terminal and run `az login` to make sure your Azure credientials are stored.

In [ ]:
import azure.quantum
workspace = azure.quantum.Workspace(
    subscription_id="bd1621e9-9094-48e4-99f4-00256be51682",
    resource_group="AzureQuantum",
    name="qir-demo",
    location="eastus",
    credential=AzureCliCredential()
)

In [ ]:
with open('./output/qutip_program_simple.bc', 'rb') as f:
    bitcode = f.read()

In [ ]:
job = azure.quantum.Job.from_input_data(
    workspace=workspace,
    name="qir-demo",
    provider_id="rigetti",
    target="rigetti.sim.qvm",
    input_data_format="qir.v1",
    output_data_format="microsoft.quantum-results.v1",
    input_data=bitcode,
    input_params={
        "shots": 1,
        "entryPoint": "main",
        "arguments": []
    }
)

In [ ]:
info = job.details.as_dict()

In [ ]:
job.get_results()

---

## 4. b) Simulating quantum programs locally

Now that we have generated our qir file, we can run it locally using [qir-runner](https://github.com/microsoft/qir-runner), a Rust based simulator that can run QIR programs.
The project is just getting started, so we included a [pre-built binary](https://github.com/microsoft/qir-runner/actions/runs/3423320625) in the repo for you to use, but you can also build it yourself from the [GitHub repo](https://github.com/microsoft/qir-runner).
Full build integration with the QIR Codespace is coming soon! <3

Since qir-runner is a command line program, we can again us the `!` magic command to run it in a Jupyter Notebook cell.
All we need to do is pass the path to the QIR file we generated in the previous step.
The runner optionally takes the name of an entry point function, which we will skip here.

In [1]:
!./runner/qir-runner ./output/qutip_program_simple.bc

RESULT	ARRAY_START
RESULT	1
RESULT	1
RESULT	ARRAY_END


We can see that we get two bits/measurement results in the same way our program submitted to Azure Quantum did!🥳